In [1]:
import pandas as pd
# never forget: https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona
import geopandas as gpd
import requests
pd.set_option('display.max_columns',None)
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from collections import defaultdict
from copy import deepcopy
import pickle
from typing import Set, Tuple, List, Dict
import random
from shapely.geometry import Point
from sklearn.neighbors import BallTree
from tqdm import tqdm

## Purpose

This notebook reads FAF5 link data and US Census county data and generates an artifact mapping:

 (lat, long) -> (STATEFP, COUNTYFP)

In [40]:
#for lat long data
TARGET_CRS = "EPSG:4326"
ALBERS_CRS = "EPSG:9822"

### Get FAF5 geographic data

In [3]:
target_path = '../data/raw/fa5_highway_links.zip'
faf5_links_gdf = gpd.read_file(target_path)

In [4]:
assert len(faf5_links_gdf['objectid'].unique())==len(faf5_links_gdf)

In [5]:
faf5_links_gdf = faf5_links_gdf.to_crs(TARGET_CRS)

#### Extract the lat/long of each link's start and end point (up to some precision)

In [6]:
%%time
# 6th decimal place is 0.11m in lat/long. IF you go to greater granularity you still get the same number of nodes
# https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude
round_to_digits = partial(round,ndigits=6)
def map_start_end_lat_long(row):
    """ Each LINESTRING object consists of lat/longs - and the bounds get the extreme endpoints"""
    long_start, lat_start = map(round_to_digits,  row.geometry.coords[0])
    long_end, lat_end = map(round_to_digits,  row.geometry.coords[-1])
    return (row.objectid, (lat_start, long_start), (lat_end, long_end))

# we iterate through the set of links in parallel, extracting formatted endpoints
edge_data = [map_start_end_lat_long(row) for row in faf5_links_gdf.itertuples()]

CPU times: total: 12.2 s
Wall time: 12.2 s


In [7]:
# from the formatted endpoints, we create unique nodes, indexed from 0
node_set: Set[Tuple[float, float]] = {(lat,long) for record in edge_data for lat,long in record[1:]}
node_dict: Dict[Tuple[float, float], int] = {node_lat_long: idx for idx, node_lat_long in enumerate(node_set)}
inv_node_dict: Dict[int, Tuple[float, float]] = {v: k for k,v in node_dict.items()}

In [8]:
print(f"Number of unique lat/longs: {len(node_set):,}")

Number of unique lat/longs: 348,495


### Get county geographic data

In [9]:
target_path = '../data/raw/us_county_shp_files.zip'
county_gdf = gpd.read_file(target_path)

In [10]:
county_gdf = county_gdf.to_crs(TARGET_CRS)

In [11]:
# county shp file data is unique at STATEFP, COUNTYFP composite key
assert len(county_gdf[['STATEFP','COUNTYFP']].drop_duplicates())==len(county_gdf)

In [87]:
# construct a map of this dataframe's index to (STATEFP,COUNTYFP)
county_idx_map = {row.Index: (row.STATEFP,row.COUNTYFP) for row in county_gdf.itertuples()}

### Compute the node -> (state, county) mapping

So, some of the highway nodes are outside of the US county system (and the FAF5 regions); we first exclude those by determining whether any network lat/longs are outside of the union of all counties.

Unfortunately, if we just test whether every county (of the 3K+) contains the lat/long in question, it takes ~20ms for each lat/long. There are ~350K unique lat/longs...so we're need 350*20=7000 seconds = 1.94 hours. Because we are lazy, it pays to limit the number of counties that we're considering.

If we can just test ~10 "close" counties, it takes ~50µs to test whether the counties contain the lat/long. So we make a ball tree based on county centroids and query the ball tree to get in the neighborhood of each lat/long. We then test whether the neighborhood contains the lat/long in question.

This takes roughly ~1.5s for testing 1000 nodes, so 350*1.5=525 seconds = 8.75 minutes

### Determine if any highway nodes are outside of the union of all counties

In [15]:
%%time
unioned_counties = county_gdf.unary_union

CPU times: total: 34.2 s
Wall time: 34.3 s


In [19]:
node_set_pdf = pd.DataFrame(node_set, columns=['latitude','longitude'])
node_set_gdf = gpd.GeoDataFrame(node_set_pdf, geometry=gpd.points_from_xy(node_set_pdf.longitude, node_set_pdf.latitude), crs=TARGET_CRS)

In [34]:
%%time
unioned_counties = unioned_counties.simplify(tolerance=1) # up to 1 m
node_within_counties = node_set_gdf.within(unioned_counties)

CPU times: total: 2min 56s
Wall time: 2min 56s


In [66]:
node_set_pdf['in_county'] = np.where(node_within_counties,True,False)
node_set_pdf['in_county'].value_counts()

in_county
True     341431
False      7064
Name: count, dtype: int64

#### Compute a ball tree based on the county centroids

In [51]:
county_centroids = [(some_point.y, some_point.x) for some_point in county_gdf.to_crs(ALBERS_CRS).centroid.to_crs(TARGET_CRS)]

In [52]:
county_centroid_lat_long_radians = np.deg2rad(np.array(county_centroids))
county_centroid_ball_tree = BallTree(county_centroid_lat_long_radians, metric='haversine')

In [89]:
%%time
faf_link_radians = np.deg2rad(np.array(list(node_set)))
distances_radians, faf_link_to_county_centroid_idx = county_centroid_ball_tree.query(faf_link_radians,k=20)

CPU times: total: 8.42 s
Wall time: 8.39 s


In [90]:
len(faf_link_to_county_centroid_idx)

348495

In [96]:
%%time
lat_long_to_county_idx, errors = dict(), list()
for idx, some_node in enumerate(node_set_pdf.itertuples()):
    if some_node.in_county==False:
        lat_long_to_county_idx[some_node.Index] = np.nan
        continue
    candidate_counties = county_gdf.iloc[faf_link_to_county_centroid_idx[idx]]
    contain_result = candidate_counties.contains(node_set_gdf.iloc[idx].geometry)
    containing_counties = contain_result.loc[contain_result]
    if len(containing_counties) != 1:
        errors.append((idx, (some_node.latitude, some_node.longitude)))
        lat_long_to_county_idx[some_node.Index] = np.nan
        continue

    containing_county = county_gdf.iloc[containing_counties.index[0]]
    lat_long_to_county_idx[some_node.Index] = (containing_county['STATEFP'], containing_county['COUNTYFP'])
    # if idx > 10000:
    #     break

CPU times: total: 10min 11s
Wall time: 10min 12s


### Serialize the results

In [98]:
# structure of this file is a dictionary with 
# key = (lat,long)
# value = Optional[(STATEFP, COUNTYFP)]: which US county this lat/long falls into

lat_long_to_county_idx_canonical = dict()
for node_idx, state_county_target in lat_long_to_county_idx.items():
    target_node = node_set_pdf.iloc[node_idx]
    lat_long_to_county_idx_canonical[(target_node.latitude,target_node.longitude)]=state_county_target

with open('../data/transformed/network_node_lat_long_to_state_county_id.pickle','wb') as fp:
    pickle.dump(lat_long_to_county_idx_canonical, fp)

In [101]:
assert len(lat_long_to_county_idx_canonical.keys())==len(node_set)

### Ad hoc investigation of some errors

Classes of errors:

1. Some lat/long happen to be on the border of 1 or more counties (exactly) so we'd have to pick which county they're allocated to. For now we just ignore them.
2. Some lat/longs are outside of the "real" counties but are within the simplified counties. We ignore these as well



In [103]:
len(errors)

123

In [106]:
rand_idx, (lat, long) = random.choice(errors)
m = county_gdf.iloc[faf_link_to_county_centroid_idx[rand_idx]].explore()
node_set_gdf.iloc[[rand_idx]].explore(m=m,color='red')

### Serialize the results

In [41]:
# we relied on the index of the county geographic data to identify each county row - but, for usage with census data,
# we serialize using the (STATEFP, COUNTYFP) code
faf_zone_id_to_state_county_id = dict()
for faf_zone_id, county_data in faf_zone_to_county_canonical.items():
    faf_zone_id_to_state_county_id[faf_zone_id] = {county_idx_map.get(k):v for k,v in county_data.items()}

# structure of this file is a dictionary with 
# key = FAF zone ID
# value = dict of (STATEFP, COUNTYFP): percentage of area of the county covered by the FAF zone
with open('../data/transformed/faf_zone_id_to_state_county_id.pickle','wb') as fp:
    pickle.dump(faf_zone_id_to_state_county_id, fp)